<a href="https://colab.research.google.com/github/Topherrrrr/mec2-projects/blob/main/Korean_Income_and_welfare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

#Files.upload produces a large amount of output. Added this in to display 1 instead, not used elsewhere
x=1
x

Saving archive.zip to archive.zip


1

In [3]:
dataset_name = 'hongsean/korea-income-and-welfare'
zip_name = dataset_name.split('/')[-1]

!kaggle datasets download -d {dataset_name}
!unzip archive.zip -d my_data

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
Archive:  archive.zip
  inflating: my_data/Korea Income and Welfare.csv  
  inflating: my_data/job_code_translated.xlsx  


In [4]:
!pip install scikeras


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
data = pd.read_csv('/content/my_data/Korea Income and Welfare.csv')
data

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,11600.0,6,1,1967,5,1,1,874,1,
92853,98000701,2015,11,5,8327.0,6,1,1967,5,1,1,874,1,
92854,98000701,2016,12,5,7931.0,6,1,1967,5,1,1,874,1,
92855,98000701,2017,13,5,8802.0,5,1,1967,5,1,1,874,1,


In [6]:
#Removing all null values where the occupation is empty

np.where(pd.isnull(data))
data = data.loc[data['occupation']!=' ']
data = data.fillna(method='ffill')  # Forward fill

data = data.dropna()  # Drop rows with missing values
data = pd.get_dummies(data)

#Scaling the data
scaler = MinMaxScaler()
data['income']=scaler.fit_transform(data[['income']])
data


,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,...,company_size_2,company_size_3,company_size_4,company_size_5,company_size_6,company_size_7,company_size_8,company_size_9,company_size_99,reason_none_worker_
8,20101,2005,1,1,0.446984,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
10,20101,2007,3,1,0.445730,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
11,20101,2008,4,1,0.448353,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
12,20101,2009,5,1,0.447713,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
13,20101,2010,6,1,0.447637,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,0.466789,6,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1
92853,98000701,2015,11,5,0.460522,6,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1
92854,98000701,2016,12,5,0.459763,6,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1
92855,98000701,2017,13,5,0.461431,5,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1


In [7]:
#Separating the X and Y variables. I want to predict the income based on the other values in the dataset

random_state = 436435
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('income', axis=1),
    data['income'], test_size=0.2, random_state=random_state
    )


In [8]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 47371 entries, 39744 to 72973
Series name: income
Non-Null Count  Dtype  
--------------  -----  
47371 non-null  float64
dtypes: float64(1)
memory usage: 740.2 KB


In [9]:
X_train.shape

(47371, 266)

In [10]:
data.columns[:]

Index(['id', 'year', 'wave', 'region', 'income', 'family_member', 'gender',
       'year_born', 'education_level', 'marriage',
       ...
       'company_size_2', 'company_size_3', 'company_size_4', 'company_size_5',
       'company_size_6', 'company_size_7', 'company_size_8', 'company_size_9',
       'company_size_99', 'reason_none_worker_ '],
      dtype='object', length=267)

In [11]:
type(y_train)

#Scaling the remaining X values in the dataset
continuous_cols = list(data)
continuous_cols.remove('income')
preprocessor = ColumnTransformer(
    transformers = [
        ('cont', MinMaxScaler(), continuous_cols)
    ])
preprocessor.fit_transform(X_train)

array([[0.37476807, 0.23076923, 0.23076923, ..., 0.        , 0.        ,
        0.        ],
       [0.25505049, 0.69230769, 0.69230769, ..., 0.        , 0.        ,
        0.        ],
       [0.07766844, 0.30769231, 0.30769231, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.15431626, 0.69230769, 0.69230769, ..., 0.        , 0.        ,
        0.        ],
       [0.5139793 , 0.23076923, 0.23076923, ..., 0.        , 0.        ,
        0.        ],
       [0.65952954, 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
#Creating and training a Sequential model
#It doesn't look like the model is learning anything. My ask is how I could dig deeper into the model to find any underlying issues?

model=Sequential()
model.add(Dense(1, activation='relu', input_shape=(266,)))
model.compile(loss='mae', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1481/1481 [==============================] - 6s 3ms/step - loss: 181224.7344 - accuracy: 0.0000e+00
Epoch 2/5
1481/1481 [==============================] - 5s 3ms/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 3/5
1481/1481 [==============================] - 6s 4ms/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 4/5
1090/1481 [=====================>........] - ETA: 1s - loss: 0.4531 - accuracy: 0.0000e+00